# Transfer Learning - VGGNet

**Załadujemy gotowy model (w tym przypadku VGGNet19) i dostroimy go do nowego zadania klasyfikacji binarnej - przykład z książki "Uczenie głębokie i sztuczna inteligencja. Interaktywny przewodnik ilustrowany"**

### Ładujemy zależności

In [1]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Ładujemy wytrenowany model VGG19

In [2]:
vgg19 = VGG19(include_top=False, #ostatnie warstwy zagęszczone, specyficzne dla zbioru, nie zostają załadowane
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

80134624/80134624 [==============================] - 5s 0us/step


### "Zamrażamy" warstwy modelu VGGNet19 - nie będą się douczać

In [3]:
for layer in vgg19.layers:
    layer.trainable = False

### Tworzymy całościowy model, "u góry" dodajemy VGGNet19, a dalej własne warstwy

**Zadanie 3. Dodaj warstwę spłaszczającą, dropout 0.5 i warstwę pozwalającą sklasyfikować binarnie obiekty**

In [4]:
model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 2)                 50178     
                                                                 
Total params: 20074562 (76.58 MB)
Trainable params: 50178 (196.01 KB)
Non-trainable params: 20024384 (76.39 MB)
_________________________________________________________________


### Kompilujemy model

In [6]:
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

### Pobierz zbiór danych z [Kaggle'a](https://www.kaggle.com/datasets/dansbecker/hot-dog-not-hot-dog) i rozpakuj

### Zbiór jest mały, więc tworzymy instancje klasy ImageDataGenerator, które pozwolą nam powiększyć dane

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

### Deklarujemy rozmiar paczki

In [8]:
batch_size = 32

### Powiększamy dane

In [9]:
train_generator = train_datagen.flow_from_directory(
    directory= './train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory= './test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


**Zadanie 4. Wypisz co oznaczają wszystkie zadeklarowane parametry generatorów danych**

tu odpowiedź

### Uczymy model z danymi z generatora (dla ułatwienia używamy do tego oddzielnej funkcji, choć będzie w następnej wersji tensorflowa usunięta)

In [10]:
model.fit_generator(train_generator, steps_per_epoch=15, epochs=16, validation_data=valid_generator, validation_steps=15)

C:\Users\kryst\AppData\Local\Temp\ipykernel_13436\2768083069.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=15, epochs=16, validation_data=valid_generator, validation_steps=15)


Epoch 1/16
15/15 [==============================] - 78s 5s/step - loss: 1.1991 - accuracy: 0.5279 - val_loss: 0.5373 - val_accuracy: 0.6979
Epoch 2/16
15/15 [==============================] - 82s 6s/step - loss: 0.6665 - accuracy: 0.6567 - val_loss: 0.6783 - val_accuracy: 0.6687
Epoch 3/16
15/15 [==============================] - 102s 7s/step - loss: 0.5543 - accuracy: 0.7511 - val_loss: 0.4879 - val_accuracy: 0.7500
Epoch 4/16
15/15 [==============================] - 100s 7s/step - loss: 0.4218 - accuracy: 0.8090 - val_loss: 0.4430 - val_accuracy: 0.7833
Epoch 5/16
15/15 [==============================] - 99s 7s/step - loss: 0.3660 - accuracy: 0.8498 - val_loss: 0.4573 - val_accuracy: 0.7750
Epoch 6/16
15/15 [==============================] - 102s 7s/step - loss: 0.3099 - accuracy: 0.8584 - val_loss: 0.4503 - val_accuracy: 0.7917
Epoch 7/16
15/15 [==============================] - 100s 7s/step - loss: 0.3410 - accuracy: 0.8369 - val_loss: 0.4337 - val_accuracy: 0.8000
Epoch 8/16
15/15